In [7]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder\
    .appName("TestJob")\
    .master("spark://spark-master:7077")\
    .getOrCreate()

In [9]:
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
df = spark.createDataFrame(data, ["Name", "Age"])

df_filtered = df.filter(df["Age"] > 28)
df_filtered.show()

+-------+---+
|   Name|Age|
+-------+---+
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [6]:
# spark.stop()